In [ ]:
import open3d as o3d
import numpy as np

from Eval.Mesh import CreatePointcloudFromDir
from Eval.Clouds.Sphere import CreateBaseSpherePointCloud
from Utils.Viz import VizualiseBaseTargetPointclouds
from Eval.Error import CalculateRMSE, CalculateSTD
from Utils.Format import MilimeterToMeter, MeterToMilimeter

In [ ]:
# Create Base Clouds for Comparison, Created Programatically for High Accuracy.

# Diameter of Base IRL Sphere: 200mm
baseSphere = CreateBaseSpherePointCloud(diameter = MilimeterToMeter(200), resolution=5000)
baseSphere.paint_uniform_color([1, 1, 1])

In [ ]:
# Current Technique and Data in Review

eval = 'Point_E' # Either Kinect, PolyCam or Point_E

evalDataDir = f'../data/{eval}/sphere.ply'

In [ ]:
# Get the Obtained Data Clouds for Comparison, on the Base Programatically Created Clouds.

collectedSphere = CreatePointcloudFromDir(evalDataDir)
collectedSphere.paint_uniform_color([1.0, 0.0, 0.0])

# Perform outlier removal on the collected sphere

#collectedSphere = collectedSphere.remove_statistical_outlier(nb_neighbors=20, std_ratio=0.05)


match eval:
    case 'Kinect':
        print("Scaling Kinect Cloud to Match Meter Unit Virtual Environment")
    
    case 'PolyCam':
        print("Scaling PolyCam Cloud to Match Meter Unit Virtual Environment")

    case 'Point_E':
        print("Scaling Point_E Cloud to Match Meter Unit Virtual Environment")
        collectedSphere.scale(MilimeterToMeter(200), collectedSphere.get_center())


In [ ]:
# Perform ICP Registration to Obtain a Transformation Matrix
# This allows for more accurate comparison of the clouds, by making them similar positioning.


icpRegResults = o3d.pipelines.registration.registration_icp(collectedSphere, 
                                                            baseSphere, 
                                                            0.1, 
                                                            np.identity(4), 
                                                            o3d.pipelines.registration.TransformationEstimationPointToPoint(), 
                                                            o3d.pipelines.registration.ICPConvergenceCriteria(max_iteration=1000))

print(icpRegResults)

# Apply the Transformation Matrix to the Obtained Data Clouds
collectedSphere.transform(icpRegResults.transformation)

In [ ]:
# Quick Visualization of both the Obtained Cloud and the Base Cloud.

#VizualiseBaseTargetPointclouds(baseSphere, collectedSphere)

#o3d.visualization.draw_geometries(
#    [baseSphere, collectedSphere],
#    window_name="Base and Target Pointclouds",
#    width=1280,
#    height=720,
#)

In [ ]:
# Calculate the RMSE between the two clouds.

rmse = CalculateRMSE(baseSphere, collectedSphere)

# Convert to mm
rmse *= 1000

print(f'RMSE (mm): {rmse}')

In [ ]:
# Calculate the STD between the two clouds.

stdData = CalculateSTD(baseSphere, collectedSphere)

# Convert to mm
stdData["std"] *= 1000
stdData["mean"] *= 1000

stdData["min"] *= 1000
stdData["max"] *= 1000
##

print(f'STD (mm): {stdData}')

### Save and Plot Data

In [ ]:
from Eval.Plotting import plot_technique_rmse, plot_technique_std
from Utils.Data import JSON

In [ ]:
rmsejsonDir = "../data/techniques-rmse-data.json"
stdjsonDir = "../data/techniques-std-data.json"

rmse_data = JSON(rmsejsonDir)
std_data = JSON(stdjsonDir)

In [ ]:
# Update Stored RMSE Data
rmse_data.data[eval]["sphere"].update({
    "rmse": rmse,
    "inlier_rmse": MeterToMilimeter(icpRegResults.inlier_rmse) # Convert to mm
})

rmse_data.print()
rmse_data.save()

# Update Stored STD Data
std_data.data[eval]["sphere"].update(stdData)

std_data.print()
std_data.save()


In [ ]:
# Create Figure Based on Current RMSE Dataset

rmseFig = plot_technique_rmse(rmse_data.data, upper_lim=12) # 1.2cm

stdFig = plot_technique_std(std_data.data, x_gap_size=1, y_gap_size=5)

rmseFig.show()
stdFig.show()

In [ ]:
# Save Figures

rmseFig.savefig("../data/techniques-rmse-diagram.png", format="png", dpi=300)
stdFig.savefig("../data/techniques-std-diagram.png", format="png", dpi=300)